In [1]:
!pip install youtube_transcript_api

In [2]:
!pip install openai python-dotenv


In [3]:
import os
import openai
from getpass import getpass

# Prompt for key (hidden input, secure)
os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")

# Initialize client using the env variable
client = openai.OpenAI(api_key=os.environ["OPENAI_API_KEY"])


Enter your OpenAI API key: ··········


In [4]:
from youtube_transcript_api import YouTubeTranscriptApi

In [5]:
def get_video_id(url_link):
    return url_link.split("v=")[-1].split("&")[0]

In [6]:
video_id = get_video_id("https://www.youtube.com/watch?v=VGFpV3Qj4as")

In [7]:
transcript = YouTubeTranscriptApi.get_transcript(video_id)

In [8]:
transcript

[{'text': 'When you start learning AI, you will',
  'start': 0.08,
  'duration': 4.719},
 {'text': 'come across so many different jargon',
  'start': 2.32,
  'duration': 5.28},
 {'text': 'that will most likely confuse you. My',
  'start': 4.799,
  'duration': 5.441},
 {'text': 'goal today is to teach you all these',
  'start': 7.6,
  'duration': 5.52},
 {'text': 'fundamental concepts in a most simple',
  'start': 10.24,
  'duration': 5.599},
 {'text': "and intuitive way. Let's get started.",
  'start': 13.12,
  'duration': 4.72},
 {'text': 'Artificial intelligence is this',
  'start': 15.839,
  'duration': 4.641},
 {'text': 'overarching field in computer science',
  'start': 17.84,
  'duration': 5.04},
 {'text': 'where we train computers to perform',
  'start': 20.48,
  'duration': 4.879},
 {'text': 'tasks which we the humans are generally',
  'start': 22.88,
  'duration': 5.08},
 {'text': 'good at. for example, recognizing',
  'start': 25.359,
  'duration': 6.561},
 {'text': 'patterns

In [9]:
transcript_text = " ".join([line['text'] for line in transcript])

In [10]:
transcript_text

'When you start learning AI, you will come across so many different jargon that will most likely confuse you. My goal today is to teach you all these fundamental concepts in a most simple and intuitive way. Let\'s get started. Artificial intelligence is this overarching field in computer science where we train computers to perform tasks which we the humans are generally good at. for example, recognizing patterns, visual capabilities, uh our voice or text capabilities and so on. Machine learning is an important subdomain within artificial intelligence. And in machine learning, there are two main branches. Statistical machine learning and deep learning. In statistical ML you will see all these statistical algorithms such as linear regression decision tree kins etc that you use for tasks such as classification regression etc. We\'ll we\'ll talk about those task more in detail going forward. But these are some of the algorithms that comes in the statistical ML. When you talk about deep lea

In [11]:
!pip install deepmultilingualpunctuation

In [12]:
from deepmultilingualpunctuation import PunctuationModel

# Load model
model = PunctuationModel()

# Restore punctuation
punctuated_text = model.restore_punctuation(transcript_text)

print(punctuated_text)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cpu
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/token_classification.py:181: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.NONE"` instead.
  warnings.warn(


When you start learning AI, you will come across so many different jargon that will most likely confuse you. My goal today is to teach you all these fundamental concepts in a most simple and intuitive way. Let's get started. Artificial intelligence is this overarching field in computer science where we train computers to perform tasks which we, the humans, are generally good at, for example, recognizing patterns, visual capabilities, uh, our voice or text capabilities, and so on. Machine learning is an important subdomain within artificial intelligence And in machine learning. there are two main branches: Statistical machine learning and deep learning. In statistical ML, you will see all these statistical algorithms, such as linear regression, decision tree, kins, etc. that you use for tasks such as classification, regression, etc. We'll we'll talk about those task more in detail going forward, But these are some of the algorithms that comes in the statistical ML. When you talk about d

In [13]:
from transformers import pipeline

summarizer = pipeline("summarization", model="google/pegasus-xsum")
summary = summarizer(punctuated_text[:1024], max_length=120, min_length=30, do_sample=False)
print(summary[0]['summary_text'])

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu


In this video, I'm going to teach you all you need to know about artificial intelligence, machine learning, deep learning, and statistical machine learning.


In [14]:
from transformers import pipeline, AutoTokenizer

# Load model and tokenizer
model_name = "google/pegasus-xsum"
summarizer = pipeline("summarization", model=model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Function to split long text into chunks of ≤ 512 tokens
def split_text_into_token_chunks(text, max_tokens=512):
    words = text.split()
    chunk = []
    current_token_count = 0

    for word in words:
        tokenized_word = tokenizer.tokenize(word)
        if current_token_count + len(tokenized_word) > max_tokens:
            yield " ".join(chunk)
            chunk = [word]
            current_token_count = len(tokenized_word)
        else:
            chunk.append(word)
            current_token_count += len(tokenized_word)

    if chunk:
        yield " ".join(chunk)

# Safely summarize each token-limited chunk
summaries = []

for chunk in split_text_into_token_chunks(punctuated_text):
    try:
        # Truncate input again at encoder level just to be sure
        summary = summarizer(
            chunk,
            max_length=120,
            min_length=30,
            do_sample=False,
            truncation=True
        )
        summaries.append(summary[0]['summary_text'])
    except Exception as e:
        print(f"⚠️ Error on chunk: {e}")

# Combine all summaries
final_summary = "\n".join(summaries)
print("\n✅ FINAL SUMMARY:\n")
print(final_summary)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu
Your max_length is set to 120, but your input_length is only 10. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=5)



✅ FINAL SUMMARY:

In this video, I'm going to show you how to learn artificial intelligence in a very simple and intuitive way, so you don't have to be a computer scientist to understand it.
In this video, I'm going to talk about statistical machine learning and spam classification, and then I'm going to show you how to train a computer program on spam and non-spam emails.
Machine learning is a discipline in computer science where we train machines on data so that they can make predictions without explicit programming, in this video I will be talking about the difference between traditional software programming and machine learning.
classification and regression are two major categories of problems that we solve using machine learning, and here I have listed a few of these uh problems, Okay, So classification and regression are two major categories of problems that we solve using machine learning, and here I have listed a few of these uh problems, Okay, So classification and regressio

In [15]:
import os
from openai import OpenAI

api_key = os.getenv("OPENAI_API_KEY")

if not api_key:
    raise ValueError("OPENAI_API_KEY environment variable not set.")

client = OpenAI(api_key=api_key)

prompt = "Your prompt goes here"

chat_completion = client.chat.completions.create(
    messages=[
        {"role": "user", "content": prompt}
    ],
    model="gpt-3.5-turbo",
    temperature=1,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
)

print(chat_completion.choices[0].message.content)




Hello! How can I assist you today?


In [16]:
print(chat_completion.choices[0].message.content)

Hello! How can I assist you today?


In [17]:
chat_completion

ChatCompletion(id='chatcmpl-Bu6W9SKJUdwv3a2zouDqhb6sCxfTm', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! How can I assist you today?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1752711121, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=9, prompt_tokens=11, total_tokens=20, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))